In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Reports

## Default Usage

By default, OpenMDAO will generate several common reports when you run your model.

1. N2 diagram
2. Scaling report
3. Coloring report

These reports, will, by default be put into a folder named after the Problem name. For example, if your model
has a single Problem and its default name, `problem1`, was not overridden, the reports will be placed in a directory named
`problem1_reports`.

The user has control over this reporting by setting some environment variables

## Turning off the reporting system

To turn off all the report generation that is part of the default report generation system of OpenMDAO,
the user should set an environment variable of `OPENMDAO_REPORTS` to any one of these three values: '0', 'false', 'off'.

## Setting the directory for the reports

If the user wishes to have the reports directory placed into a different directory other than the current working
directory, the user can set the environment variable, `OPENMDAO_REPORTS_DIR` to the name of that directory. The
directory does not have to exist. It will be created by the reporting system.

## Support for subproblems

The reporting system supports subproblems. For example, a model has two Problems and they have the default names of
`problem1` and `problem2`, the reporting system will put the reports into the folders, `problem1_reports` and
`problem2_reports`.

## Adding user defined reports

The user can register a user defined report with the reporting system.

Here is a script to show how this is done. Note that the function that is called to do the reporting must have
as its first argument, the `Problem`.

After this script is run, there will be a file named `user_report.txt` in the directory `problem1_reports`.

In [ ]:
from openmdao.utils.reports import register_report
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

def user_defined_report(prob):
    with open('user_report.txt', "w") as f:
        f.write(f"Do some reporting on the Problem, {prob._name}\n")

register_report(user_defined_report, 'user defined report', 'setup', 'pre')

prob = om.Problem()
model = prob.model

model.add_subsystem('p1', om.IndepVarComp('x', 0.0), promotes=['x'])
model.add_subsystem('p2', om.IndepVarComp('y', 0.0), promotes=['y'])
model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])

model.add_design_var('x', lower=0.0, upper=1.0)
model.add_design_var('y', lower=0.0, upper=1.0)
model.add_objective('f_xy')

prob.driver = om.ScipyOptimizeDriver()

prob.setup()
prob.run_driver()
prob.cleanup()